---
# Introduction

---

**Problem Statement:**

* Whether out at a restaurant or buying tickets to a concert, modern life counts on the convenience of a credit card to make daily purchases. It saves us from carrying large amounts of cash and also can advance a full purchase that can be paid over time.
* How do card issuers know we’ll pay back what we charge? That’s a complex problem with many existing solutions—and even more potential improvements, to be explored in this competition.
* Credit default prediction is central to managing risk in a consumer lending business. 
 * Credit default prediction allows lenders to optimize lending decisions, which leads to a better customer experience and sound business economics. 
* Current models exist to help manage risk. But it's possible to create better models that can outperform those currently in use.

* The objective of this competition is to predict the probability that a customer does not pay back their credit card balance amount in the future based on their monthly customer profile. 
 * The target binary variable is calculated by observing 18 months performance window after the latest credit card statement, and if the customer does not pay due amount in 120 days after their latest statement date it is considered a default event.

---
**Importing Libraries:**
* To get started we will use Python for data pre-processing and model building.
* Import python libraries as necessary to get started for data load and later import other libraries as needed
---

In [ ]:
import numpy as np 
# data processing, CSV file I/O 
import pandas as pd 
# data processing, CSV file I/O
import dask.dataframe as dd
# module finds all the pathnames matching a specified pattern
import glob 
import os
# importing pyplot interface using matplotlib
import matplotlib.pyplot as plt 
# importing seaborn library for interactive visualization
import seaborn as sns 
# Importing WordCloud for text data visualization
from wordcloud import WordCloud
# importing matplotlib for plots
import matplotlib
# importing datetime for using datetime
from datetime import datetime
# importing plotly for interactive plots
import plotly.express as px
# importing missingno for missing value plot
import missingno as msno
# importing regex library for use of regex
import re
# importing Counter for counting :)
from collections import Counter

import plotly.graph_objs as go

In [ ]:
# importing SimpleImputer for handling missing value
from sklearn.impute import SimpleImputer
# importing MissingIndicator for handling missing value
from sklearn.impute import MissingIndicator
# importing StandardScaler for standardization
from sklearn.preprocessing import StandardScaler
# importing OnHotEncoder for encoding categorical variable
from sklearn.preprocessing import OneHotEncoder
# importing for transformation
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
# importing PCA for handling dimensonality reduction
from sklearn.decomposition import PCA

# importing pipeline for chaining model building activities
#from sklearn.pipeline import Pipeline
#from sklearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline as mp
# importing FeatureUnion for combining transformers
from sklearn.pipeline import FeatureUnion

# importing samplers for handling data imbalance
from imblearn.combine import SMOTEENN 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler 
from imblearn.under_sampling import RandomUnderSampler 

# importing train_test_split for train and validation split
from sklearn.model_selection import train_test_split
# importing SelectFromModel to select features from model 
from sklearn.feature_selection import SelectFromModel               

In [ ]:
# importing classifiers to try with
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# importing metrics required for model evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# importing RepeatedKFold for cross validation
from sklearn.model_selection import RepeatedKFold
# importing for model evaluation
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
# importing RepeatedStratifiedKFold for model evaluation
from sklearn.model_selection import RepeatedStratifiedKFold
# importing GridSearchCV for hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from yellowbrick.model_selection import ValidationCurve

---
# Dataset Load
---

**Dataset:**

* **train_data.csv** - training data with multiple statement dates per customer_ID
* **train_labels.csv** - target label for each customer_ID
* **test_data.csv** - corresponding test data; objective is to predict the target label for each customer_ID
* **sample_submission.csv** - a sample submission file in the correct format

---

Let us check size of dataset CSV file

In [ ]:
# calculate file size in KB, MB, GB
def convert_bytes(size):
    """ Convert bytes to KB, or MB or GB"""
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            return "%3.1f %s" % (size, x)
        size /= 1024.0

# display CSV file with size
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        csvfile=os.path.join(dirname, filename)
        csvfilesize = os.path.getsize(csvfile)
        filesize = convert_bytes(csvfilesize)
        print(f'{csvfile} size is', filesize, 'bytes')

/kaggle/input/amex-default-prediction/sample_submission.csv size is 59.1 MB bytes
/kaggle/input/amex-default-prediction/train_data.csv size is 15.3 GB bytes
/kaggle/input/amex-default-prediction/test_data.csv size is 31.5 GB bytes
/kaggle/input/amex-default-prediction/train_labels.csv size is 29.3 MB bytes


In [ ]:
from pathlib import Path

input_path = Path('/kaggle/input/amex-default-prediction/')

---
Considering large number of rows around 5.5 million in **train_data.csv** dataset, using nrows option to load first 100k rows from dataset file for Model building.

---

---
Will try to leverage outcome based on EDA done so far

https://www.kaggle.com/code/girishkumarsahu/american-express-default-prediction-eda

---

Load train_data.csv dataset file using nrows=100000

In [ ]:
# Loading dataset train_data.csv
train_df_sample = pd.read_csv('../input/amex-default-prediction/train_data.csv', nrows=100000)

In [ ]:
# get shape of dataframe
print('Shape of dataset is:', train_df_sample.shape)

# print summary of dataframe
train_df_sample.info()

Shape of dataset is: (100000, 190)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 190 entries, customer_ID to D_145
dtypes: float64(185), int64(1), object(4)
memory usage: 145.0+ MB


**Observations:**

* There are total 190 variables in train_data.csv dataset
    * There are 185 variables(Columns) as dtype float64, 1 variable(Column) as dtype int64 and 4 variables(Columns) as dtype object

---
Need to load **train_labels.csv** for customer_ID with target label as 1 for Default and 0 for Not Default

---

In [ ]:
# Loading dataset train_labels.csv
train_label_df = pd.read_csv('../input/amex-default-prediction/train_labels.csv')

In [ ]:
# get shape of dataframe
print('Shape of dataset is:', train_label_df.shape)

# print summary of dataframe
train_label_df.info()

Shape of dataset is: (458913, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458913 entries, 0 to 458912
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_ID  458913 non-null  object
 1   target       458913 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.0+ MB


**Observations:**

* There are total 458,913 entries for target label with customer_ID
* There is variable (column) customer_ID which has dtype as object and variable (column) target which has dtype as int64

---
Using nrows option to load first 100k rows from **test_data.csv** dataset file. Set customer_ID as index.

---

In [ ]:
# Loading dataset test_data.csv
test_df = pd.read_csv('../input/amex-default-prediction/test_data.csv', nrows=100000, index_col='customer_ID')

In [ ]:
# get shape of dataframe
print('Shape of dataset is:', test_df.shape)

# print summary of dataframe
#test_df.info(verbose=True)
test_df.info()

Shape of dataset is: (100000, 189)
<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7 to 0246c7eb137ed9b08014d66e29caf1772b0512becef11a1eda0948b8b8908576
Columns: 189 entries, S_2 to D_145
dtypes: float64(185), int64(1), object(3)
memory usage: 145.0+ MB


**Observation:**

* There are 185 variables(Columns) as dtype float64, 1 variable(Column) as dtype int64 and 4 variables(Columns) as dtype object, same structure as train_data.csv

---
Need to merge train_labels dataset with train_data dataset for target label.

---

In [ ]:
# Merge of train_df_sample and train_label_df dataframe using key as customer_ID
train_df = pd.merge(train_df_sample, train_label_df, how="inner", on=["customer_ID"])

In [ ]:
# print summary of merged dataframe
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Columns: 191 entries, customer_ID to target
dtypes: float64(185), int64(2), object(4)
memory usage: 146.5+ MB


Drop customer_ID and S_2 variable in train_data and drop S_2 variable in test_data which are not required for ML model building

In [ ]:
#drop customer_ID and S_2 from train_df dataframe which are not required for model building
train_df.drop(axis=1, columns=['customer_ID','S_2'], inplace=True)

In [ ]:
#drop S_2 in test_df dataframe which is not required for model building
test_df.drop(axis=1, columns=['S_2'], inplace=True)

---
**Q: Is there any duplicate row in train dataset sample?**

---

In [ ]:
#check if any duplicate row
if (any(train_df.duplicated())):
    print("Yes")
else:
    print("No")

No


---
**Q: Is there any duplicate row in test dataset sample?**

---

In [ ]:
#check if any duplicate row in test dataset
if (any(test_df.duplicated())):
    print("Yes")
else:
    print("No")

No


---
**Q: Are there any missing value in train dataset sample?**

---

In [ ]:
# Check for missing value
if(any(train_df.isna().sum())):
    print("Yes")
else:
    print("No")

Yes


---
**Q: Are there any missing value in train dataset sample?**

---

In [ ]:
# Check for missing value in test dataset
if(any(test_df.isna().sum())):
    print("Yes")
else:
    print("No")

Yes


---
# Data Preprocessing

---

---
**Handle Variable with Missing Value**

---

---
Drop Variables with Missing Value (>=75%) in train dataset

---

In [ ]:
#drop variables with missing values >=75% in the train dataframe
i=0
for col in train_df.columns:
    if (train_df[col].isnull().sum()/len(train_df[col])*100) >=75:
        print("Dropping column", col)
        train_df.drop(labels=col,axis=1,inplace=True)
        i=i+1
        
print("Total number of columns dropped in train dataframe", i)

Dropping column D_42
Dropping column D_49
Dropping column D_66
Dropping column D_73
Dropping column D_76
Dropping column R_9
Dropping column B_29
Dropping column D_87
Dropping column D_88
Dropping column D_106
Dropping column R_26
Dropping column D_108
Dropping column D_110
Dropping column D_111
Dropping column B_39
Dropping column B_42
Dropping column D_132
Dropping column D_134
Dropping column D_135
Dropping column D_136
Dropping column D_137
Dropping column D_138
Dropping column D_142
Total number of columns dropped in train dataframe 23


---
Drop Variables with Missing Value (>=75%) in test dataset

---

In [ ]:
#drop variables with missing values >=75% in the test dataframe
i=0
for col in test_df.columns:
    if (test_df[col].isnull().sum()/len(test_df[col])*100) >=75:
        print("Dropping column", col)
        test_df.drop(labels=col,axis=1,inplace=True)
        i=i+1
        
print("Total number of columns dropped in test dataframe", i)

Dropping column D_42
Dropping column D_49
Dropping column D_66
Dropping column D_73
Dropping column D_76
Dropping column R_9
Dropping column B_29
Dropping column D_87
Dropping column D_88
Dropping column D_106
Dropping column R_26
Dropping column D_108
Dropping column D_110
Dropping column D_111
Dropping column B_39
Dropping column B_42
Dropping column D_132
Dropping column D_134
Dropping column D_135
Dropping column D_136
Dropping column D_137
Dropping column D_138
Dropping column D_142
Total number of columns dropped in test dataframe 23


Converting categorical variable dtype from float64 to object in train and test dataset

In [ ]:
#convert dtype for B categorical variable to object
train_df = train_df.astype({"B_30": 'str', "B_38": 'str'})
#convert dtype for B categorical variable to object
test_df = test_df.astype({"B_30": 'str', "B_38": 'str'})
#convert dtype for D categorical variable to object
train_df = train_df.astype({"D_114": 'str', "D_116": 'str', "D_117": 'str', "D_120": 'str', "D_126": 'str', "D_68": 'str'})
#convert dtype for D categorical variable to object
test_df = test_df.astype({"D_114": 'str', "D_116": 'str', "D_117": 'str', "D_120": 'str', "D_126": 'str', "D_68": 'str'})

Separate independent and dependent variable for train dataframe

In [ ]:
# separate X and y for further processing
X = train_df.drop(columns='target')
y = train_df['target']

In [ ]:
print("Shape of X", X.shape)

Shape of X (100000, 165)


In [ ]:
print("Shape of y", y.shape)

Shape of y (100000,)


Separate Categorical and Numerical variables (columns) for train dataframe

In [ ]:
# define categorical variables (columns)
categorical = list(X.select_dtypes('object').columns)
print(f"Categorical variables (columns) are: {categorical}")

# define numerical variables (columns)
numerical = list(X.select_dtypes('number').columns)
print(f"Numerical variables (columns) are: {numerical}")

Categorical variables (columns) are: ['D_63', 'D_64', 'D_68', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126']
Numerical variables (columns) are: ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'P_4', 'D_74', 'D_75', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'S_18', 'D_86', 'R_17', 'R_18', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_

---
**Handle Categorical Variable (Column)**

---

In [ ]:
# define categorical pipeline
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=np.nan)),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ('scaler', StandardScaler())
])

print(cat_pipe)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('scaler', StandardScaler())])


---
**Handle Numerical Variable (Column)**

---

In [ ]:
# define numerical pipeline
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=np.nan)),
    ('scaler', StandardScaler())
])
print(num_pipe)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())])


Combine Categorical and Numerical Pipeline Steps

In [ ]:
# combine categorical and numerical pipeline
preprocess = ColumnTransformer([
    ('cat', cat_pipe, categorical),
    ('num', num_pipe, numerical)
])

print(preprocess)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['D_63', 'D_64', 'D_68', 'B_30', 'B_38',
                                  'D_114', 'D_116', 'D_117', 'D_120',
                                  'D_126']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('scaler', StandardScaler())]),
                                 

Separate training and validation set for train dataframe

In [ ]:
# splitting training data into training and testing (validation) set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

In [ ]:
print("Shape of X_train", X_train.shape)

Shape of X_train (75000, 165)


In [ ]:
print("Shape of X_test", X_test.shape)

Shape of X_test (25000, 165)


In [ ]:
print("Shape of y_train", y_train.shape)

Shape of y_train (75000,)


In [ ]:
print("Shape of y_test", y_test.shape)

Shape of y_test (25000,)


---
# Model Building/Evaluation

---

Amex Evaluation Metric for reference

In [ ]:
# please refer sample notebook provided under competition page for details
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()
    
    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [ ]:
# function for display of model training and validation score
def model_score(model_name):
    print("#######################################################################")
    print("Training and Evaluation using", model_name)
    print("#######################################################################")
    print("preprocess - Categorical: Missing value Impute, OneHotEncoding and Scaling")
    print("preprocess - Numerical: Missing value Impute and Scaling")
    print("###########################################################################")
    model = pipe.fit(X_train, y_train)  
    print("#######################################################################")
    print (model)
    print("#######################################################################")
    print("model training score: %.3f" % pipe.score(X_train, y_train))
    print("model validation score: %.3f" % pipe.score(X_test, y_test))
    print("#######################################################################")
    print("Amex Evaluation Metric - Training: %.3f"% amex_metric(pd.DataFrame(y_train), pd.DataFrame(pipe.predict(X_train), columns=['prediction'])))
    print("Amex Evaluation Metric - Validation: %.3f"% amex_metric(pd.DataFrame(y_test), pd.DataFrame(pipe.predict(X_test), columns=['prediction'])))
    print("#######################################################################")

In [ ]:
# function for display of cross validation score
def model_cross_validation_score(model_name):
    print("#######################################################################")
    print("Training and Evaluation with Cross Validation using",model_name)
    print("#######################################################################")
    # using scoring with classification metrics
    scoring = ['accuracy', 'precision', 'recall','f1','roc_auc']
    #using RepeatedStratifiedKFold as cross validator
    cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=42)
    # cross validation returning both train and test score
    scores = cross_validate(pipe, X, y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score=True,return_estimator=True)
    print('Training Score: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f},f1-score: {:.2f}, ROC AUC: {:.2f}'.format(np.mean(scores['train_accuracy']),np.mean(scores['train_precision']), np.mean(scores['train_recall']), np.mean(scores['train_f1']), np.mean(scores['train_roc_auc'])))
    print('Validation Score: Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f},f1-score: {:.2f}, ROC AUC: {:.2f}'.format(np.mean(scores['test_accuracy']),np.mean(scores['test_precision']), np.mean(scores['test_recall']), np.mean(scores['test_f1']), np.mean(scores['test_roc_auc'])))
    print("#######################################################################")

In [ ]:
# function for display of model score via RandomizedSearchCV
def model_random_search_score(model_name):
    print("#######################################################################")
    print("Training and Evaluation with RandomizedSearchCV using",model_name)
    print("#######################################################################")
    random_search.fit(X_train,y_train)
    model = random_search.best_estimator_
    score = random_search.best_score_
    print ("Best Estimator for", model_name,"is", model,"with best score as",score)
    print("#######################################################################")
    print("Amex Evaluation Metric - Training: %.3f"% amex_metric(pd.DataFrame(y_train), pd.DataFrame(model.predict(X_train), columns=['prediction'])))
    print("Amex Evaluation Metric - Validation: %.3f"% amex_metric(pd.DataFrame(y_test), pd.DataFrame(model.predict(X_test), columns=['prediction'])))
    print("#######################################################################")

---
Using pipeline steps for Model Building, Training & Evaluation

---

In [ ]:
# pipeline steps required for model building,training and evaluation
steps = [
        ('preprocess', preprocess),
        ('over_sampler',SMOTE(random_state = 42)),
        ('under_sampler',RandomUnderSampler()),
        ('feature_selection', SelectFromModel(RandomForestClassifier(n_estimators = 10, random_state = 42, n_jobs = -1))),
        ('dimension_reduction', PCA(n_components='mle',random_state = 42)),
        ('model_estimator', RandomForestClassifier(random_state = 42))
    ]
pipe = Pipeline(steps, verbose=True)

* Using RandomForestClassifier

In [ ]:
# using custom function to display model training and validation score
model_score("RandomForestClassifier")

#######################################################################
Training and Evaluation using RandomForestClassifier
#######################################################################
preprocess - Categorical: Missing value Impute, OneHotEncoding and Scaling
preprocess - Numerical: Missing value Impute and Scaling
###########################################################################
[Pipeline] ........ (step 1 of 6) Processing preprocess, total=   2.8s
[Pipeline] ...... (step 2 of 6) Processing over_sampler, total=  10.5s
[Pipeline] ..... (step 3 of 6) Processing under_sampler, total=   0.1s
[Pipeline] . (step 4 of 6) Processing feature_selection, total=  10.0s
[Pipeline]  (step 5 of 6) Processing dimension_reduction, total=   0.4s
[Pipeline] ... (step 6 of 6) Processing model_estimator, total= 2.0min
#######################################################################
Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
        

* Using XGBClassifier

In [ ]:
# using XGBClassifier
pipe.set_params(model_estimator=XGBClassifier())
# using custom function to display model training and validation score
model_score("XGBClassifier")

#######################################################################
Training and Evaluation using XGBClassifier
#######################################################################
preprocess - Categorical: Missing value Impute, OneHotEncoding and Scaling
preprocess - Numerical: Missing value Impute and Scaling
###########################################################################
[Pipeline] ........ (step 1 of 6) Processing preprocess, total=   2.5s
[Pipeline] ...... (step 2 of 6) Processing over_sampler, total=   9.4s
[Pipeline] ..... (step 3 of 6) Processing under_sampler, total=   0.1s
[Pipeline] . (step 4 of 6) Processing feature_selection, total=  10.2s
[Pipeline]  (step 5 of 6) Processing dimension_reduction, total=   0.4s
[Pipeline] ... (step 6 of 6) Processing model_estimator, total=  56.8s
#######################################################################
Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                 

---
Using XGBClassifier with Cross-Validation and scoring with accuracy, precision, recall, f1-score and ROC AUC

---

In [ ]:
#using custom function to display cross validation score
model_cross_validation_score("XGBClassifier")

#######################################################################
Training and Evaluation with Cross Validation using XGBClassifier
#######################################################################
Training Score: Accuracy: 0.94, Precision: 0.82, Recall: 0.97,f1-score: 0.89, ROC AUC: 0.99
Validation Score: Accuracy: 0.89, Precision: 0.73, Recall: 0.89,f1-score: 0.80, ROC AUC: 0.96
#######################################################################


* Using LGBMClassifier

In [ ]:
# using LGBMClassifier
pipe.set_params(model_estimator=LGBMClassifier())
# using custom function to display model training and validation score
model_score("LGBMClassifier")

#######################################################################
Training and Evaluation using LGBMClassifier
#######################################################################
preprocess - Categorical: Missing value Impute, OneHotEncoding and Scaling
preprocess - Numerical: Missing value Impute and Scaling
###########################################################################
[Pipeline] ........ (step 1 of 6) Processing preprocess, total=   2.4s
[Pipeline] ...... (step 2 of 6) Processing over_sampler, total=   9.2s
[Pipeline] ..... (step 3 of 6) Processing under_sampler, total=   0.1s
[Pipeline] . (step 4 of 6) Processing feature_selection, total=   9.5s
[Pipeline]  (step 5 of 6) Processing dimension_reduction, total=   0.3s
[Pipeline] ... (step 6 of 6) Processing model_estimator, total=   2.4s
#######################################################################
Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                

---
Using LGBMClassifier with Cross-Validation and scoring with accuracy, precision, recall, f1-score and ROC AUC

---

In [ ]:
#using custom function to display cross validation score
model_cross_validation_score("LGBMClassifier")

#######################################################################
Training and Evaluation with Cross Validation using LGBMClassifier
#######################################################################
Training Score: Accuracy: 0.88, Precision: 0.70, Recall: 0.93,f1-score: 0.80, ROC AUC: 0.96
Validation Score: Accuracy: 0.86, Precision: 0.67, Recall: 0.91,f1-score: 0.77, ROC AUC: 0.95
#######################################################################


* Using CatBoostClassifier

In [ ]:
# using CatBoostClassifier
pipe.set_params(model_estimator=CatBoostClassifier(iterations=3,learning_rate=1,depth=6))
# using custom function to display model training and validation score
model_score("CatBoostClassifier")

#######################################################################
Training and Evaluation using CatBoostClassifier
#######################################################################
preprocess - Categorical: Missing value Impute, OneHotEncoding and Scaling
preprocess - Numerical: Missing value Impute and Scaling
###########################################################################
[Pipeline] ........ (step 1 of 6) Processing preprocess, total=   2.4s
[Pipeline] ...... (step 2 of 6) Processing over_sampler, total=   9.0s
[Pipeline] ..... (step 3 of 6) Processing under_sampler, total=   0.1s
[Pipeline] . (step 4 of 6) Processing feature_selection, total=   9.6s
[Pipeline]  (step 5 of 6) Processing dimension_reduction, total=   0.2s
0:	learn: 0.3881795	total: 106ms	remaining: 211ms
1:	learn: 0.3682953	total: 138ms	remaining: 69.2ms
2:	learn: 0.3571750	total: 169ms	remaining: 0us
[Pipeline] ... (step 6 of 6) Processing model_estimator, total=   0.5s
#######################

---
Using CatBoostClassifier with Cross-Validation and scoring with accuracy, precision, recall, f1-score and ROC AUC

---

In [ ]:
#using custom function to display cross validation score
model_cross_validation_score("CatBoostClassifier")

#######################################################################
Training and Evaluation with Cross Validation using CatBoostClassifier
#######################################################################
Training Score: Accuracy: 0.83, Precision: 0.61, Recall: 0.89,f1-score: 0.72, ROC AUC: 0.92
Validation Score: Accuracy: 0.83, Precision: 0.61, Recall: 0.88,f1-score: 0.72, ROC AUC: 0.92
#######################################################################


In [ ]:
# param_grid = dict(model_estimator=[XGBClassifier(),LGBMClassifier()])
# grid_search = GridSearchCV(pipe, param_grid=param_grid, scoring='roc_auc',cv=3,verbose=3, n_jobs=-1).fit(X,y)

In [ ]:
#estimator which gave higher score
#grid_search.best_estimator_

* Using RandomizedSearchCV for XGBClassifier

In [ ]:
# using parameters for RandomizedSearchCV for XGBClassifier
param_random = dict(model_estimator=[XGBClassifier()],model_estimator__learning_rate= [0.05,0.10,0.15,0.20,0.25,0.30],model_estimator__max_depth= [ 3, 4, 5, 6, 8, 10, 12, 15],model_estimator__min_child_weight=[ 1, 3, 5, 7 ], model_estimator__gamma=[ 0.0, 0.1, 0.2 , 0.3, 0.4 ], model_estimator__colsample_bytree =[ 0.3, 0.4, 0.5 , 0.7 ])
random_search = RandomizedSearchCV(pipe, param_distributions=param_random, n_iter=1, cv=3, scoring='roc_auc', verbose=3,random_state=42)

Best Estimator for XGBClassifier for this training sample

In [ ]:
#using custom function to display best estimator and score
model_random_search_score("XGBClassifier")

#######################################################################
Training and Evaluation with RandomizedSearchCV using XGBClassifier
#######################################################################
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Pipeline] ........ (step 1 of 6) Processing preprocess, total=   1.6s
[Pipeline] ...... (step 2 of 6) Processing over_sampler, total=   4.3s
[Pipeline] ..... (step 3 of 6) Processing under_sampler, total=   0.1s
[Pipeline] . (step 4 of 6) Processing feature_selection, total=   6.3s
[Pipeline]  (step 5 of 6) Processing dimension_reduction, total=   0.2s
[Pipeline] ... (step 6 of 6) Processing model_estimator, total=  15.7s
[CV 1/3] END model_estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gp

* Using RandomizedSearchCV for LGBMClassifier

In [ ]:
# using parameters for RandomizedSearchCV for LGBMClassifier
param_random = dict(model_estimator=[LGBMClassifier()],model_estimator__num_leaves= [20,40,60,80,100],model_estimator__min_child_samples= [5,10,15],model_estimator__max_depth=[-1,5,10,20], model_estimator__learning_rate=[0.05,0.1,0.2], model_estimator__reg_alpha =[0,0.01,0.03])
random_search = RandomizedSearchCV(pipe, param_distributions=param_random, n_iter=1, cv=3, scoring='roc_auc', verbose=3,random_state=42)

Best Estimator for LGBMClassifier for this training sample

In [ ]:
#using custom function to display best estimator and score
model_random_search_score("LGBMClassifier")

#######################################################################
Training and Evaluation with RandomizedSearchCV using LGBMClassifier
#######################################################################
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Pipeline] ........ (step 1 of 6) Processing preprocess, total=   1.6s
[Pipeline] ...... (step 2 of 6) Processing over_sampler, total=   4.3s
[Pipeline] ..... (step 3 of 6) Processing under_sampler, total=   0.1s
[Pipeline] . (step 4 of 6) Processing feature_selection, total=   6.3s
[Pipeline]  (step 5 of 6) Processing dimension_reduction, total=   0.2s
[Pipeline] ... (step 6 of 6) Processing model_estimator, total=   5.0s
[CV 1/3] END model_estimator=LGBMClassifier(), model_estimator__learning_rate=0.05, model_estimator__max_depth=10, model_estimator__min_child_samples=5, model_estimator__num_leaves=100, model_estimator__reg_alpha=0;, score=0.946 total time=  18.0s
[Pipeline] ........ (step 1 of 6) Processing preproce

---
**Prediction/Sample Submission file**

---

In [ ]:
test_df_new=test_df.reset_index()

In [ ]:
del test_df

In [ ]:
X_test_predict = test_df_new.groupby('customer_ID').tail(1)

In [ ]:
X_test_predict.shape

(8154, 166)

In [ ]:
X_test_predict.set_index('customer_ID', inplace=True)

Prediction on test dataset 

In [ ]:
model = random_search.best_estimator_

In [ ]:
# prediction on test dataset
y_test_pred = model.predict(X_test_predict)

Generation of submission.csv file with customer_ID and prediction as header

In [ ]:
# generate submission file
output = pd.DataFrame({'customer_ID': X_test_predict.index,'prediction': y_test_pred})
output.to_csv('submission.csv', index=False, header=True)

In [ ]:
output.head(600)

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1
...,...,...
595,002b04bc284c6f177ba5ea9cbda2acf386f3f63e9b4b0c...,0
596,002b0f54b8c73ee987d39f12b8935ee539618769907c1d...,0
597,002b28c047310823468c7c51ad4a94b3e69abcba06b2c3...,1
598,002b2ec09090b45c471332644d5dc9dc9f7d5dc7fb8a2a...,0


---
# Summary

---

* Variables (Columns) with missing value >= 75% was removed
* SimpleImputer was used for Variables (Columns) with missing value <= 25%
* OneHotEncoder was used for encoding Categorical Variables
* SMOTE was used to address Data Imbalance along with RandomUnderSampler
* RandomForestClassifier was used for Feature Selection
* PCA was used for Dimensionality Reduction

LGBMClassifier has been the fastest classifier on this 100k sample training dataset

---
# Next Steps

---

* **For Submission**
    * Learning usage of Dask for reading large size CSV file
    * Using Dask DataFrame and Dask ML API
    * Minimum ~925000 unique test data points needed for prediction in submission file
    * Generate submission.csv file with customer_ID and prediction as header

---
**Thank you and Happy Learning.**

---